# Summary

The products dataframe was corrupted and needs cleaning. \
We have decided the following:
1. price column had corrupted data with the following issues that were fixed:
    - Some values had two decimal points. The first point got removed.
    - The values with two decimal points were also 100x the correct value. We divided by 100.
2. promo_price is unnecessary and will be removed. We will determine discounted price based on orderlines.unit_price
3. We will add the brands to products dataframe as separating it is redundant for our purposes.


### Imports

In [ ]:
import pandas as pd

# products.csv ##################################
url = "https://drive.google.com/file/d/1khUA0GrhC4cdAgg7-Zn3RX2vI9dcxWMa/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

# brands.csv ####################################
url = "https://drive.google.com/file/d/1yBGBdJlpIjel_9Yp5HJWO_mBLPv39fx5/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands = pd.read_csv(path)

# products_clean.csv ####################################
url = "https://drive.google.com/file/d/1RfVACMhoaTl3j8gV9ZdyVWZV_TlfZgx_/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cleaned = pd.read_csv(path)

# Price

It is clear that the price column has been corrupted and carries values with two decimal points and an extra three digits. \
It is unclear *why* however the pattern seems to be predictable. We will clean and reassess.

### First we apply all our standard cleaning

In [ ]:
products = products.drop_duplicates(subset = "sku") # Drop duplicates
products = products.loc[~products['price'].isna()] # Drop rows with missing prices

### Then we have a look at the sample

In [ ]:
products.sample(20)

,sku,name,desc,price,promo_price,in_stock,type
1969,PAC0779,Samsung SSD 850 expansion kit PRO 500GB + OWC ...,500GB SSD expansion kit + Adapter Mount Pro fo...,285.98,2.405.843,1,1433
13306,OTT0148,OtterBox iPhone Case Symmetry 7/8 Clear Clear,resistant and anti thin sheath bevelled edges ...,34.99,22.99,0,11865403
2289,TRA0013,Transcend JetDrive Lite 350 64GB Macbook Pro R...,MLC memory card 64GB for Macbooks Retina 15 in...,46,429.901,0,57445397
17243,AP20292,Like new - Apple 38mm Sport Black Belt,Durable and flexible strap sports with clip cl...,59,389.995,0,2449
2399,DEV0011,Devolo Wifi Repeater 24 GHz,Amplifier Wi-Fi N 24 GHz Devolo.,399.905,329.846,1,1334
10694,SYN0128,Synology DS416 NAS server Mac and PC,Synology NAS server 4 bays and DLNA Media Center.,385.99,3.849.893,0,12175397
2857,STA0023,Startech Adapter Kit Mini DisplayPort / VGA / ...,Mini DisplayPort adapter kit / HDMI / VGA / US...,86.99,69.99,0,1325
13043,BNQ0048,"BL2420U BenQ Monitor 236 ""4K 100% sRGB pivotable",Monitor for design professionals 4K resolution...,399.99,398.989,0,1296
14769,QNA0198,QNAP TS-831X Server Nas | 8GB RAM,8-bay NAS server for business environment with...,930.49,9.299.903,0,12175397
2579,APP1004,"Apple MacBook Pro Retina 15 ""i7 22GHz | 16GB R...",New MacBook Pro 15-inch Retina screen Core i7 ...,2969,27.610.046,0,1282


### Then we apply our methods to normalize the data

In [ ]:
# Where 'price' contains double dots
double_dot_entries = products.price.astype(str).str.contains("\d+\.\d+\.\d+")

# Remove decimals, convert to int, divide by 100000, and round to two decimal places
products.loc[double_dot_entries, 'price'] = (products.loc[double_dot_entries, 'price'].str.replace('.', '')  # remove decimals
                                                                                                        .astype(int)  # convert to integer
                                                                                                        / 100000).round(2)  # divide by 100000 and round to two decimal places
products.loc['price'] = pd.to_numeric(products['price'])


<ipython-input-66-073b0a22a024>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  products.loc[double_dot_entries, 'price'] = (products.loc[double_dot_entries, 'price'].str.replace('.', '')  # remove decimals


In [ ]:
# Now we check our work
products.sample(20)

,sku,name,desc,price,in_stock,type
12886,PAC2043,Synology DS216 + II | Seagate 4TB IronWolf for...,Synology DS216 + II with 8GB of RAM and 20TB (...,170.79,0.0,12175397
12679,MOP0084,Mophie Power Reserve 2600 mAh battery 1X White...,Laptop Battery 2600mAh (18A) with LED and micr...,39.95,0.0,1515
17679,IHE0020-A,Open - iHealth Box 50 Reagent Strips glucímetros,Blood glucose test strips in the iHealth glucí...,19.95,0.0,1298
15736,APP2050,Open - Apple iPhone 7128 GB Black - Like New,Apple iPhone 7 Free 128 GB Black (MN922QL / A),879,0.0,1716
2924,KEN0220,Kensingnton Minisaver security Cable for Mac a...,Cable ultralight Mac and PC security.,69.99,0.0,1325
1954,PAC0717,Samsung SSD 850 expansion kit EVO 1TB + 8GB RA...,Upgrade kit 1TB SSD + Datadoubler Mac Mini 200...,605.96,1.0,1433
359,MUV0045,Muvit iPhone in September matte Screen Savers ...,Set 2 screen protectors for iPhone matte finis...,8.99,0.0,13555403
11706,DRO0009-A,(Open) Data Robotics Drobo 5D USB 3.0 Thunderb...,Mac and PC RAID 5-bay Thunderbolt USB 3.0.,749,0.0,1298
12996,SPH0008-A,(Open) SPRK Sphero Robotic Transparent Sphere,programmable robot for IOS SPRK,149.95,0.0,1298
18128,NTE0131,NewerTech NuGuard KX X-treme iPhone Case Red 8/7,Protective cover for massive drops NuGuard KX ...,60.99,0.0,11865403


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10535 entries, 0 to price
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       10534 non-null  object 
 1   name      10534 non-null  object 
 2   desc      10527 non-null  object 
 3   price     10534 non-null  object 
 4   in_stock  10534 non-null  float64
 5   type      10484 non-null  object 
dtypes: float64(1), object(5)
memory usage: 834.2+ KB


### Analysis

We can see by combing the samples that we have successfully fixed the issue and the prices are now usable.
As this makes up a large portion of our overall data, this was almost definitely worth the effort.

# Promo_price

The promo_price column has been corrupted. While it is possible to fix it, we can take more \
concrete payment data from orderlines.unit_price so we will remove this column entirely.


In [ ]:
products.drop('promo_price', axis=1, inplace=True)

# Brands

We will add the brands columns to the products dataframes to make exploration easier.

In [ ]:
# merge the products and brands dataframes
merged_df = pd.merge(products, brands, left_on=products['sku'].str[:3], right_on=brands['short'], how='left')

# drop the redundant short column as it isn't necessary. The three letters are already supplied in the sku column
merged_df.drop(['key_0', 'short'], axis=1, inplace=True)

# rename the 'long' column to brand
merged_df.rename(columns={'long': 'brand'}, inplace=True)


# Download

In [ ]:
 #from google.colab import files

 #products.to_csv("Cleaned_products.csv", index=False)
 #files.download("Cleaned_products.csv")

# Testing Sample

Check that the data makes sense.

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10535 entries, 0 to price
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       10534 non-null  object 
 1   name      10534 non-null  object 
 2   desc      10527 non-null  object 
 3   price     10534 non-null  object 
 4   in_stock  10534 non-null  float64
 5   type      10484 non-null  object 
dtypes: float64(1), object(5)
memory usage: 576.1+ KB


In [ ]:
products_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10534 entries, 0 to 10533
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       10534 non-null  object 
 1   name      10534 non-null  object 
 2   desc      10527 non-null  object 
 3   price     10534 non-null  float64
 4   in_stock  10534 non-null  int64  
 5   type      10484 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 493.9+ KB


In [ ]:
products.sample(50)

,sku,name,desc,price,in_stock,type
1869,PAC0623,Samsung SSD 850 expansion kit EVO 500GB + OWC ...,500GB SSD expansion kit + Adapter Mount Pro fo...,215.98,1.0,1433
2697,IFX0031,iFixit PATA adapter 127mm optical bay Macbook ...,Replacement tray optical bay hard drive for Ma...,39.95,0.0,12755395
13063,GTE0137,G-Technology G-DRIVE Mobile Hard Drive 1TB USB...,external hard disk 1TB aluminum housing USB co...,106.99,1.0,11935397
13793,APP1838,"Apple MacBook Pro 13 ""Core i5 with Touch Bar 3...",New MacBook Pro 13-inch Core i5 Touch Bar to 3...,2119,0.0,2158
12325,AII0037-A,"(Open) Aiino housing Retina MacBook Pro 15 ""Tr...",Retina MacBook Pro casing 15 inch matte finish.,39.99,0.0,1298
2961,LIF0076,Lifeproof LifeActiv Bracelet + adapter QuickMount,IPhone sports armband.,49.99,0.0,5405
7825,PAC1626,"Apple iMac 27 ""Core i7 Retina 5K 4GHz | 8GB RA...",IMac desktop computer 27 inch Retina 5K 4GHz C...,4669,0.0,"5,74E+15"
2933,MUV0163,"Muvit Bracelet Universal Fino 4-47 ""Lima + Spo...",sporty and comfortable armband for iPhone 5 / ...,19.95,0.0,5405
2025,PHI0052,Philips Hue Bloom separate light lamp,LED lamp Hue color system works with app,59.95,1.0,11905404
17416,AP20305,Like new - Apple iPhone 32GB Rose Gold,Refurbished 32GB iPhone rose gold color,419,0.0,51601716
